<a href="https://colab.research.google.com/github/gokulaspires92/Multi_lingual_chat_bot/blob/main/Chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Streamlit (Fixes: ModuleNotFoundError: No module named 'streamlit')
!pip install streamlit numpy

# Install FAISS, the core RAG vector library (using the stable CPU version)
!pip install faiss-cpu

# Install document parsing for .docx files (Fixes: Document import)
!pip install python-docx

# Install the Sentence Transformer model library (for embeddings)
!pip install sentence-transformers

# Install localtunnel to expose the Streamlit web app
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
changed 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import faiss
from docx import Document
import nltk
from nltk.tokenize import sent_tokenize
import logging
import os
import sys
import time
# We must import SentenceTransformer for local RAG
from sentence_transformers import SentenceTransformer
import re
import json

# Suppress warnings
logging.getLogger("transformers").setLevel(logging.ERROR)

# --- Streamlit Configuration ---
# Switching to a slightly smaller model (L12-v2) to improve loading stability.
st.set_page_config(page_title="HCL GUVI Stable RAG Chatbot (English Only)", layout="wide")

# --- Global Configuration ---
DOCUMENT_PATH = "Chatbot.docx"
# Using the slightly faster/smaller 'all-MiniLM-L12-v2' model instead of L6-v2 for robustness
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"

# --- Utility Functions ---

def split_text_into_chunks(raw_text):
    """Splits raw text into sentence chunks for indexing."""

    # Simple sentence tokenizer using regex
    sentences = re.split(r'(?<=[.!?])\s+', raw_text)
    sentences = [s.strip() for s in sentences if s.strip()]

    if not sentences:
        try:
            nltk.download('punkt', quiet=True)
        except:
            pass # Ignore download errors if already present

        sentences = sent_tokenize(raw_text)
        sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

# --- Caching and Initialization Functions ---
@st.cache_resource(show_spinner=False)
def initialize_knowledge_base():
    """Initializes the FAISS index, text chunks, and embedding model with visual progress."""

    status_container = st.container()
    progress_bar = status_container.progress(0, text="Initializing...")

    try:
        # 1. Load and process document
        progress_bar.progress(10, text="1/4: Extracting text from DOCX...")

        document = Document(DOCUMENT_PATH)
        raw_text = "\n".join([paragraph.text for paragraph in document.paragraphs]).lower()

        # 2. Tokenize into sentences (chunks)
        progress_bar.progress(30, text="2/4: Tokenizing document into sentences...")
        sentences = split_text_into_chunks(raw_text)

        if not sentences:
            status_container.error(f"The document at '{DOCUMENT_PATH}' is empty or could not be processed.")
            return None, None, None

        status_container.info(f"Knowledge base loaded with {len(sentences)} sentences.")

        # 3. Load Embedding Model (L12-v2)
        progress_bar.progress(50, text="3/4: Downloading/Loading Sentence Transformer model (L12-v2)...")
        embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

        # 4. Create Embeddings and FAISS Index
        progress_bar.progress(70, text="4/4: Creating vector index (FAISS)...")
        embeddings = embedding_model.encode(sentences, convert_to_tensor=False)
        embeddings = np.array(embeddings).astype('float32')

        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)

        progress_bar.progress(100, text="Initialization Complete!")
        status_container.success("Knowledge base (FAISS index) built successfully! App is ready.")
        time.sleep(1)
        status_container.empty()

        return index, sentences, embedding_model
    except Exception as e:
        print(f"UNHANDLED EXCEPTION in initialize_knowledge_base: {e}", file=sys.stderr)
        progress_bar.progress(0, text="Initialization Failed.")
        status_container.error(f"A critical error occurred during knowledge base initialization. Error: {e}")
        return None, None, None

# --- Core Local RAG Logic ---

def retrieve_context(query_embedding, index, sentences, k=3):
    """Retrieves the top k most relevant sentences from the FAISS index."""
    try:
        query_vector = np.array(query_embedding).astype('float32').reshape(1, -1)
        if index is None or index.ntotal == 0:
            return ""

        D, I = index.search(query_vector, k)
        context = [sentences[i] for i in I[0] if i < len(sentences)]
        return "\n".join(context)
    except Exception as e:
        print(f"Error during context retrieval: {e}", file=sys.stderr)
        return ""

def generate_local_rag_response(prompt: str, context: str):
    """
    Generates a response using simple, local, template-based logic
    (Fallback for no local LLM/Q&A model).
    """
    if not context:
        return "I am sorry, I couldn't find any relevant information in the knowledge document to answer your question."

    response = (
        f"Based on the knowledge document, here is the relevant context for your query:\n\n"
        f"**Question:** *{prompt}*\n\n"
        f"**Relevant Context Found:**\n> {context.replace('\n', '\n> ')}\n\n"
        f"*(This simple RAG system provides the source context directly, as it avoids using a separate local LLM/Q&A model to meet environmental constraints.)*"
    )
    return response

# --- Main Chatbot Application ---
def run_chatbot(index, sentences, embedding_model):
    """The main Streamlit application logic."""
    st.markdown("""
        <div style='text-align: center; background-color: #f0f2f6; padding: 10px; border-radius: 10px;'>
            <h1 style='color: #4a4a4a; font-size: 1.8em; margin: 0;'>
                HCL GUVI Stable RAG Chatbot (English Only)
            </h1>
            <p style='color: #6c757d; margin: 0;'>
                Trained on the <b>Chatbot.docx</b> document using local vector search.
            </p>
        </div>
        """, unsafe_allow_html=True)

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []
        st.session_state.messages.append({"role": "assistant", "content": "Hello! I am the HCL GUVI Knowledge Bot. I can answer questions about the document, but please ask in **English**."})

    # Display chat messages from history on app rerun
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # We are replacing the problematic st.chat_input with a reliable st.text_input.
    # The key='input_key' ensures the input is persistent across runs.
    prompt = st.text_input("Ask a question in English (e.g., 'What is GUVI?')", key='input_key', on_change=handle_input)

    # We use a button to explicitly trigger the action, which is more reliable in this environment.
    st.button("Send Query", on_click=handle_input)

    # Logic is handled by the callback function to prevent running the entire script multiple times
    # when text_input state changes, but we still need to process the prompt if it exists.
    if st.session_state.get('new_prompt'):
        process_prompt(st.session_state['new_prompt'], index, sentences, embedding_model)
        st.session_state['new_prompt'] = None # Clear the prompt after processing

def handle_input():
    """Handles the user input when Enter is pressed or the button is clicked."""
    # Check if the text input box has content and hasn't been processed yet
    if st.session_state['input_key'] and st.session_state['input_key'] != st.session_state.get('last_prompt', ''):
        st.session_state['new_prompt'] = st.session_state['input_key']
        st.session_state['last_prompt'] = st.session_state['input_key']
        st.session_state['input_key'] = '' # Clear the input box immediately

def process_prompt(prompt, index, sentences, embedding_model):
    """Processes the user's query and generates a response."""
    if prompt:
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Processing (Local RAG)..."):

                # a. Embed the query
                query_embedding = embedding_model.encode([prompt], convert_to_tensor=False)[0]

                # b. Retrieve the context
                context = retrieve_context(query_embedding, index, sentences)

                # c. Generate the local response
                final_response = generate_local_rag_response(prompt, context)

                # Display the final response
                st.markdown(final_response)

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": final_response})


# --- Application Entry Point ---
if not os.path.exists(DOCUMENT_PATH):
    st.error(f"FATAL ERROR: The required document '{DOCUMENT_PATH}' was not found. Please ensure it is uploaded.")
else:
    # Initialize knowledge base (the only component that requires local loading)
    index, sentences, embedding_model = initialize_knowledge_base()

    # Run application if successful
    if index is not None and embedding_model is not None:
        run_chatbot(index, sentences, embedding_model)
    else:
        st.error("Application could not fully initialize. Check the logs.")


Writing app.py


In [ ]:
!pkill -f streamlit
!streamlit run app.py --server.port 8501 & sleep 3 && npx localtunnel --port 8501
'''3.  When the application loads, you should see the **input box and the button** directly on the main page. Type your question and click **"Send Query"**.'''




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.68.62.88:8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋your url is: https://wild-cobras-win.loca.lt
2025-09-30 13:10:17.580669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759237817.649671   11708 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759237817.662985   11708 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759237817.714735   11708 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759237817.714819   1